In [1]:
!pip install -q transformers datasets peft accelerate evaluate bitsandbytes

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
from datasets import load_dataset

dataset = load_dataset("glue", "sst2")
dataset = dataset["train"].train_test_split(test_size=0.2)

def preprocess(example):
    return tokenizer(example["sentence"], truncation=True, padding="max_length", max_length=128)

encoded_dataset = dataset.map(preprocess, batched=True)
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


Map:   0%|          | 0/53879 [00:00<?, ? examples/s]

Map:   0%|          | 0/13470 [00:00<?, ? examples/s]

In [4]:
from transformers import TrainingArguments, Trainer
import evaluate
import torch

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=-1)
    return accuracy.compute(predictions=preds, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=8
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=encoded_dataset["test"],
    compute_metrics=compute_metrics
)

print("🔍 Evaluating Pretrained Model...")
original_results = trainer.evaluate()


🔍 Evaluating Pretrained Model...


In [5]:
from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "v_lin"],  
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()


The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


In [6]:
training_args = TrainingArguments(
    output_dir="./peft_output",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    evaluation_strategy="epoch",
    save_strategy="no",
    logging_dir="./logs"
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    compute_metrics=compute_metrics
)

print("🚀 Training PEFT Model...")
trainer.train()


C:\Users\elist\anaconda3\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


🚀 Training PEFT Model...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.045200,0.036388,0.989235


TrainOutput(global_step=6735, training_loss=0.04597153015816398, metrics={'train_runtime': 16199.3695, 'train_samples_per_second': 3.326, 'train_steps_per_second': 0.416, 'total_flos': 1814906125390848.0, 'train_loss': 0.04597153015816398, 'epoch': 1.0})

In [7]:
peft_model.save_pretrained("./distilbert_peft_lora")
tokenizer.save_pretrained("./distilbert_peft_lora")


('./distilbert_peft_lora\\tokenizer_config.json',
 './distilbert_peft_lora\\special_tokens_map.json',
 './distilbert_peft_lora\\vocab.txt',
 './distilbert_peft_lora\\added_tokens.json',
 './distilbert_peft_lora\\tokenizer.json')

In [8]:
from peft import PeftModel, PeftConfig

peft_config = PeftConfig.from_pretrained("./distilbert_peft_lora")

base_model = AutoModelForSequenceClassification.from_pretrained(peft_config.base_model_name_or_path)
loaded_peft_model = PeftModel.from_pretrained(base_model, "./distilbert_peft_lora")

In [9]:
trainer = Trainer(
    model=loaded_peft_model,
    args=training_args,
    eval_dataset=encoded_dataset["test"],
    compute_metrics=compute_metrics
)

print("📊 Evaluating Fine-Tuned PEFT Model...")
peft_results = trainer.evaluate()

print(f"\nOriginal Accuracy: {original_results['eval_accuracy']:.4f}")
print(f"Fine-Tuned Accuracy: {peft_results['eval_accuracy']:.4f}")

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


📊 Evaluating Fine-Tuned PEFT Model...



Original Accuracy: 0.9891
Fine-Tuned Accuracy: 0.9892
